### 7.15 레이블링, 물체 구분

In [9]:
import cv2
import numpy as np

#1
src = cv2.imread('c:/data/circles.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, res = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)

#2
# 픽셀이 0 이면 배경, 0이 아니면 물체
ret, labels = cv2.connectedComponents(res)
print('ret= ', ret)

vMin, vMax, lMin, lMax = cv2.minMaxLoc(labels)
print(vMin, vMax, lMin, lMax)

# 0은 배경 3은 물체
print(labels[335:345, 330:340])

#3
dst = np.zeros(src.shape, dtype=src.dtype)
for i in range(1, ret):
    r = np.random.randint(256)
    g = np.random.randint(256)
    b = np.random.randint(256)
    dst[labels==i] = [b, g, r]

cv2.imshow('res', res)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

ret=  4
0.0 3.0 (0, 0) (339, 338)
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 3]
 [0 0 0 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3 3 3 3]]
